Read the csv file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.offline import plot
import datetime

pd.options.mode.chained_assignment = None
df = pd.read_csv('/kaggle/input/earthquake-database/database.csv')

Check for incomplete data

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.heatmap(df.isnull(), ax=ax)
plt.grid()
plt.show()

Check for some wrong format date data based on lenght

In [ ]:
l = df['Date'].str.len()
print(l.value_counts())
ind = np.where([l == 24])[1]
print('Indices with corrupted data:', ind)

isolate Date length = 24 to df1 and convert it

In [ ]:
df1 = df.loc[ind]
df1['DateTime'] = pd.to_datetime(df1['Date']).dt.tz_convert(None)

convert the date time in df without df1, create a new column based on Date and Time

In [ ]:
df = df.drop(ind)
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time']).dt.time
df['DateTime'] = [datetime.datetime.combine(a, b) for a, b in zip(df['Date'], df['Time'])]

concat df and df1 an drop 2 columns and check dtype: datetime64[ns]

In [ ]:
df = pd.concat([df, df1])
df = df.drop(['Date', 'Time'], axis=1)
print(df['DateTime'])

visualise DateTime per year

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns.countplot(ax=ax, x=df['DateTime'].dt.year)
plt.show()

Pair ploting

In [ ]:
sns.pairplot(data=df)
plt.show()

Static map figure - total earthquakes

In [ ]:
import configparser
config = configparser.ConfigParser()
config = config.read('config.ini')
mapbox_token = 'pk.eyJ1IjoiY2xhdWNvdCIsImEiOiJja2h4MTIxd2UwMzNzMnlvNzVycXN2dW14In0.J_KSzOY3YGDzozRdHvMJgQ'

fig = go.Figure(
    go.Scattermapbox(
        lat=df['Latitude'],
        lon=df['Longitude'],
        mode='markers',
        marker = go.scattermapbox.Marker(
            size=df['Magnitude'],
            colorscale='IceFire',
            color=df['Magnitude'],
            showscale=True,
            colorbar={'title': 'Magnitude', 'titleside': 'top', 'thickness': 4, 'ticksuffix': ''},
        ),
            customdata=np.stack(
                (pd.Series(df.DateTime.dt.year),
                 df['Magnitude'],
                 df['Depth']),
                axis=-1
                ),
            hovertemplate="""
                <extra></extra>
                <em>%{customdata[0]}  </em><br>
                Magnitude  %{customdata[1]}<br>
                Depth  %{customdata[2]}<br>
                """,

    )
)

# Specify layout information
fig.update_layout(
    mapbox=dict(
        accesstoken=mapbox_token, #
        center=go.layout.mapbox.Center(lat=25, lon=142),
        zoom=1
    )
)

Animated figure - framed per year and play button

In [ ]:
df['Y'] = df['DateTime'].dt.year
df['M'] = df['DateTime'].dt.month
df = df.set_index(['Y', 'M'])
index_list = df.index.levels[0].tolist()

frames=[{
        'name': 'frame_{}'.format(i),

        'data': [{
            'type': 'scattermapbox',
            'lat': df.xs(i)['Latitude'],
            'lon': df.xs(i)['Longitude'],
            'marker': go.scattermapbox.Marker(
                size=df.xs(i)['Magnitude']+(df.xs(i)['Magnitude']-4)**3,
                color=df.xs(i)['Magnitude'],
                showscale=True,
                colorscale='IceFire',
                colorbar={'title': 'Magnitude', 'titleside': 'top', 'thickness': 4, 'ticksuffix': ' Mgn'}
                ),
        'customdata': np.stack((
            df.xs(i)['Magnitude'],
            df.xs(i)['Depth'],
            pd.Series(df.xs(i).index)),
                axis=-1
                ),
        'hovertemplate': "<extra></extra><em> Magnitude  %{customdata[0]}<br>Depth  %{customdata[1]}",
        }],
    } for i in index_list]

sliders = [{
            'transition':{'duration': 0},
            'x':0.08,
            'len':0.88,
            'currentvalue':{'font':{'size':15}, 'prefix':'📅 ', 'visible':True, 'xanchor':'center'},
            'steps':[
                {
                    'label':i,
                    'method':'animate',
                    'args':[
                        ['frame_{}'.format(i)],
                        {'mode':'immediate', 'frame':{'duration':1000, 'redraw': True}, 'transition':{'duration':500, }}
                      ],
                } for i in index_list]
        }]

play_button = [{
        'type':'buttons',
        'showactive':True,
        'x':0.045, 'y':-0.08,
        'buttons':[{
            'label':'🎬', # Play
            'method':'animate',
            'args':[
                None,
                {
                    'frame':{'duration':1000, 'redraw':True},
                    'transition':{'duration':500},
                    'fromcurrent':True,
                    'mode':'immediate',
                }
            ]
        }]
    }]

# Defining the initial state
data = frames[0]['data']

# Adding all sliders and play button to the layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=play_button,
    mapbox={
        'accesstoken':'pk.eyJ1IjoiY2xhdWNvdCIsImEiOiJja2h4MTIxd2UwMzNzMnlvNzVycXN2dW14In0.J_KSzOY3YGDzozRdHvMJgQ',
        'center':{"lat": 25, "lon": 142},
        'zoom':1,
        'style':'light',
    }
)

# Creating the figure
fig = go.Figure(data=data, layout=layout, frames=frames)

# Display the figure
fig.show()